# Synth. Obs.: Analytic disk

We create synthetic observations for the Magritte model of the analytic spiral that was created in the [this example](../0_creating_models/0_create_analytic_disk.ipynb).

## Setup

Import the required functionalty.

In [1]:
import magritte.core     as magritte   # Core functionality
import magritte.plot     as plot       # Plotting
import magritte.tools    as tools      # Save fits

from astropy import units              # Unit conversions

Define a working directory (you will have to change this). We assume here that the scripts of the [this example](../0_creating_models/0_create_analytic_disk.ipynb) have already been executed and go back to that working directory.

In [2]:
wdir = "/home/frederik/Magritte-examples/Analytic_disk/"

Define file names.

In [3]:
model_file = f'{wdir}model_analytic_disk.hdf5'   # Analytic spiral Magritte model

Load the Magritte model.

In [4]:
model = magritte.Model(model_file)

                                           
-------------------------------------------
  Reading Model...                         
-------------------------------------------
 model file = /home/frederik/Magritte-examples/Analytic_disk/model_analytic_disk.hdf5
-------------------------------------------
Reading parameters...
Reading points...
Reading rays...
Reading boundary...
Reading chemistry...
Reading species...
Reading thermodynamics...
Reading temperature...
Reading turbulence...
Reading lines...
Reading lineProducingSpecies...
Reading linedata...
read num 1
read sym CO
nlev = 41
nrad = 1
Reading collisionPartner...
Reading collisionPartner...
Reading quadrature...
Reading radiation...
Reading frequencies...
npoints = 75847
nlines  = 1
nquads  = 51
Not using scattering!
                                           
-------------------------------------------
  Model read, parameters:                  
-------------------------------------------
  npoints    = 75847
  nrays      =

## Model the medium

Initialize the model by setting up a spectral discretisation, computing the inverse line widths and initializing the level populations with their LTE values.

In [5]:
model.compute_spectral_discretisation ()
model.compute_inverse_line_widths     ()
model.compute_LTE_level_populations   ()

Computing spectral discretisation...
Computing inverse line widths...
Computing LTE level populations...


0

In this example we will work with the LTE level populations and **do not demand** statistical equilibrium.

In [6]:
# Iterate level populations until statistical equilibrium
# model.compute_level_populations (False, 1)

## Make synthetic observations

Now we can make synthetic observations of the model.

In [7]:
fcen = model.lines.lineProducingSpecies[0].linedata.frequency[0]
vpix = 1.0e+3   # velocity pixel size [m/s] 
dd   = vpix * (model.parameters.nfreqs()-1)/2 / magritte.CC
fmin = fcen - fcen*dd
fmax = fcen + fcen*dd

# Ray orthogonal to image plane
ray_nr = 0

model.compute_spectral_discretisation (fmin, fmax)
model.compute_image                   (ray_nr)

Computing spectral discretisation...


0

Computing image...
--------------------------------------


## Plot observations

Plot the resulting channel maps with matplotlib.

In [8]:
import matplotlib                                   # Mpl
import numpy              as np                     # Data structures
import matplotlib.pyplot  as plt                    # Plotting

from matplotlib.gridspec  import GridSpec           # Plot layout
from astropy              import constants, units   # Unit conversions
from scipy.interpolate    import griddata           # Grid interpolation
from palettable.cubehelix import cubehelix2_16      # Nice colormap
from tqdm                 import tqdm               # Progress bars
from ipywidgets           import interact           # Interactive plots


image_nr =  -1
zoom     = 2.0
npix_x   = 100
npix_y   = 100
x_unit   = units.au
v_unit   = units.km/units.s
method   = 'nearest'
    
# Extract data of last image
imx = np.array(model.images[image_nr].ImX)
imy = np.array(model.images[image_nr].ImY)
imI = np.array(model.images[image_nr].I)
imv = np.array(model.radiation.frequencies.nu)[0]

Imin = np.min(np.nan_to_num(imI, nan=1.0e+99))
imI  = np.nan_to_num(imI, nan=Imin)

# Extract the number of frequency bins
nfreqs = model.parameters.nfreqs()
    
# Set image boundaries
x_min, x_max = np.min(imx)/zoom, np.max(imx)/zoom
y_min, y_max = np.min(imy)/zoom, np.max(imy)/zoom

# Create image grid values
xs = np.linspace(x_min, x_max, npix_x)
ys = np.linspace(y_min, y_max, npix_y)
    
# Extract the spectral / velocity data
freqs = np.array(model.radiation.frequencies.nu)[0]
f_ij  = np.mean(freqs)
velos = (freqs - f_ij) / f_ij * constants.c.to(v_unit).value

# Interpolate the scattered data to an image (regular grid)
Is = np.zeros((nfreqs))
zs = np.zeros((nfreqs, npix_x, npix_y))
for f in range(nfreqs):
    # Nearest neighbor interpolate scattered image data
    zs[f] = griddata(
        (imx, imy),
        imI[:,f],
        (xs[None,:], ys[:,None]),
        method=method
    )
    Is[f] = np.sum(zs[f])
Is = Is / np.max(Is)

zs_min = 10.0*np.min(zs)
zs_max =  1.0*np.max(zs)
   
figs = []
gs   = GridSpec(1,2, wspace=.1, width_ratios=[2, 1])

for f in tqdm(range(nfreqs)):
    fig = plt.figure(dpi=300)
    ax1 = fig.add_subplot(gs[0])
    ax1.contourf(
        xs / (1.0 * x_unit).si.value,
        ys / (1.0 * x_unit).si.value,
        zs[f],
        cmap=cubehelix2_16.mpl_colormap,
        vmin=zs_min,
        vmax=zs_max,
        levels=250
    )
    ax1.set_aspect('equal')
    ax1.set_xlabel(f'image x [{x_unit}]', labelpad = 10)
    ax1.set_ylabel(f'image y [{x_unit}]', labelpad = 10)
    
    ax2 = fig.add_subplot(gs[1])
    ax2.plot(velos, Is/np.max(Is))
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    ax2.axvline(velos[f], c='red')
    ax2.set_ylabel(f'Relative intensity',  labelpad=15)
    ax2.set_xlabel(f'velocity [{v_unit}]', labelpad=10)
    asp = 2*np.diff(ax2.get_xlim())[0] / np.diff(ax2.get_ylim())[0]
    ax2.set_aspect(asp)
    
    figs.append(fig)
  
    plt.close()
    
# Create a widget for plots
widget = interact(lambda v: figs[v], v=(0, len(figs)-1))

100%|██████████████████████████████████████████████████| 51/51 [00:07<00:00,  6.85it/s]


interactive(children=(IntSlider(value=25, description='v', max=50), Output()), _dom_classes=('widget-interact'…

(The plot is only interactive in a live notebook.)

Save the image cube in a fits file.

In [9]:
tools.save_fits(model)

Created image directory: /home/frederik/Magritte-examples/Analytic_disk/images/
Written file to: /home/frederik/Magritte-examples/Analytic_disk/images/image.fits
